In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import re
import nltk

In [2]:
train = pd.read_csv("../input/sentiment-analysis-msa-phase-2/train.csv")
train.dropna(subset = ['text'], inplace = True)

In [3]:
train.text = train.text.apply(lambda x: re.sub(r'\(?http\S+', '', x))
train.text = train.text.apply(lambda x: re.sub(r'[^\w\s]', '', x))

In [4]:
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
train.text = train.text.apply(lambda x: ' '.join([word for word in nltk.word_tokenize(x) if word.lower() not in stopwords]))

In [5]:
lemmatizer = nltk.WordNetLemmatizer()
train.text = train.text.apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in nltk.word_tokenize(x)]))

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn import metrics

In [7]:
vectorizer = CountVectorizer()

In [8]:
train.dropna(subset=['text'], inplace=True)
X = vectorizer.fit_transform(train.text)

In [9]:
nb = MultinomialNB()

#nb.fit(features_matrix, item_we_want_to_predict)
nb.fit(X, train.sentiment)

MultinomialNB()

In [10]:
test = pd.read_csv('../input/sentiment-analysis-msa-phase-2/test.csv')
test.dropna(subset=['text'], inplace=True)
test.head(10)

,textID,text
0,f87dea47db,Last session of the day http://twitpic.com/67ezh
1,96d74cb729,Shanghai is also really exciting (precisely -...
2,eee518ae67,"Recession hit Veronique Branquinho, she has to..."
3,01082688c6,happy bday!
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!
5,726e501993,that`s great!! weee!! visitors!
6,261932614e,I THINK EVERYONE HATES ME ON HERE lol
7,afa11da83f,"soooooo wish i could, but im in school and my..."
8,e64208b4ef,and within a short time of the last clue all ...
9,37bcad24ca,What did you get? My day is alright.. haven`...


In [11]:
test.text = test.text.apply(lambda x: re.sub(r'[^\w\s]', '', x) )

In [12]:
test.text = test.text.apply(lambda x: ' '.join([word for word in nltk.word_tokenize(x) if word.lower() not in stopwords]))

In [13]:
test.text = test.text.apply(lambda x: re.sub(r'\(?http\S+', '', x))

In [14]:
lemmatizer = nltk.WordNetLemmatizer()
test.text = test.text.apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in nltk.word_tokenize(x)]))

In [15]:
test_count = vectorizer.transform(test.text)
predicted = nb.predict(test_count)

In [16]:
predicted

array(['positive', 'positive', 'negative', ..., 'negative', 'positive',
       'neutral'], dtype='<U8')

In [17]:
test['sentiment'] = predicted

In [18]:
test.head()

,textID,text,sentiment
0,f87dea47db,Last session day,positive
1,96d74cb729,Shanghai also really exciting precisely skyscr...,positive
2,eee518ae67,Recession hit Veronique Branquinho quit compan...,negative
3,01082688c6,happy bday,positive
4,33987a8ee5,like,neutral


In [19]:
test = test.drop(['text'], axis = 1)
test.head()

,textID,sentiment
0,f87dea47db,positive
1,96d74cb729,positive
2,eee518ae67,negative
3,01082688c6,positive
4,33987a8ee5,neutral


In [20]:
test.to_csv("submission.csv", index = False)